In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data import dataloader
from neuralnet import nn
import pixiedust_node

Pixiedust database opened successfully


In [3]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

pixiedust_node 0.2.4 started. Cells starting '%%node' may contain Node.js code.
Data loaded from Firebase!


---

In [9]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))

none: 19 on-target samples in 2 iteration blocks
environment: 106 on-target samples in 4 iteration blocks
body: 101 on-target samples in 2 iteration blocks
cup: 33 on-target samples in 3 iteration blocks
food: 35 on-target samples in 4 iteration blocks
nails: 69 on-target samples in 3 iteration blocks
smoke: 92 on-target samples in 3 iteration blocks
eyebrow: 235 on-target samples in 5 iteration blocks
nose: 249 on-target samples in 4 iteration blocks
above-ear: 403 on-target samples in 5 iteration blocks
behind-ear: 220 on-target samples in 4 iteration blocks
opposite-cheek: 74 on-target samples in 6 iteration blocks
chin: 166 on-target samples in 1 iteration block
cheek: 686 on-target samples in 6 iteration blocks
forehead: 176 on-target samples in 1 iteration block
top-head: 160 on-target samples in 1 iteration block
back-head: 170 on-target samples in 1 iteration block
opposite-face: 381 on-target samples in 2 iteration blocks
paint-mouth: 81 on-target samples in 3 iteration blocks

---

In [4]:
#train
cheek_train = nn.define_trainer_data(
    pilot_data,
    {
        "target": ["cheek"],
        "offtarget": ["environment"]
    },
    [
        "thermopile1",
        "thermopile2",
        "thermopile3",
        "thermopile4"
    ],
    [
        1, 2, 3
    ]
)

#test true
cheek_test_true = nn.define_activation(
    pilot_data,
    [
        "cheek"
    ],
    [
        "thermopile1",
        "thermopile2",
        "thermopile3",
        "thermopile4"
    ],
    [
        2, 3
    ]
)

#test_false
cheek_test_false = nn.define_activation(
    pilot_data,
    [
        "environment"
    ],
    [
        "thermopile1",
        "thermopile2",
        "thermopile3",
        "thermopile4"
    ],
    [
        2, 3
    ]
)

---

In [5]:
%%node
var synaptic = require('synaptic');
var Neuron = synaptic.Neuron,
	Layer = synaptic.Layer,
	Network = synaptic.Network,
	Trainer = synaptic.Trainer,
	Architect = synaptic.Architect;

In [6]:
%%node
function Perceptron(input, hidden, output){
	// create the layers
	var inputLayer = new Layer(input);
	var hiddenLayer = new Layer(hidden);
	var outputLayer = new Layer(output);

	// connect the layers
	inputLayer.project(hiddenLayer);
	hiddenLayer.project(outputLayer);

	// set the layers
	this.set({
		input: inputLayer,
		hidden: [hiddenLayer],
		output: outputLayer
	});
}

// extend the prototype chain
Perceptron.prototype = new Network();
Perceptron.prototype.constructor = Perceptron;

function runPerceptronOnList(perceptron, listOfInputs){
    var outputs = [];
    for(var i=0; i<listOfInputs.length; i++){
        outputs.push(perceptron.activate(listOfInputs[i]));
    }
    return(outputs);
}

In [7]:
%%node
var perceptron5 = new Perceptron(4,5,1);
var perceptron7 = new Perceptron(4,7,1);
var trainer5 = new Trainer(perceptron5);
var trainer7 = new Trainer(perceptron7);

trainer5.train(cheek_train);
trainer7.train(cheek_train);

... ... ... ...
... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...
... ... ..... ..... ... ...


In [8]:
%%node
print(runPerceptronOnList(perceptron5, cheek_test_true))

[[0.008357277421641107], [0.008357277423129701], [0.008357277423897623], [0.008357277413185376], [0.008357277412990487], [0.008357277414183387], [0.008357277414336867], [0.008357277422750721], [0.008357277415182766], [0.008357277413890814], [0.00835727741665101], [0.00835727741808906], [0.008357277413454935], [0.008357277421436332], [0.00835727742167116], [0.008357277412521109], [0.008357277413743865], [0.008357277422955075], [0.008357277412714423], [0.008357277414124877], [0.008357277420926838], [0.008357277415191114], [0.008357277421835922], [0.0083572774205757], [0.008357277415567167], [0.008357277413867952], [0.008357277425572556], [0.008357277417463097], [0.008357277415948716], [0.008357277417196638], [0.008357277420103474], [0.00835727741671153], [0.008357277416163891], [0.008357277422286804], [0.008357277420679708], [0.008357277417166282], [0.008357277418522165], [0.008357277417912432], [0.008357277415621488], [0.008357277415189296], [0.008357277418004521], [0.008357277420723865